In [6]:
%run TEST/WZX/Test/send_email

StatementMeta(, , -1, Finished, Available)

请设置receiver_email、subject、body，以发送邮件


In [3]:
# 导包
from notebookutils import mssparkutils  
from openpyxl import load_workbook
import pandas as pd
import os
from datetime import datetime,timedelta    

StatementMeta(aasparkpool001, 898, 4, Finished, Available)

In [4]:
# 挂载blob
mssparkutils.fs.mount(
    "wasbs://bosch-dw-integration-layer@proddataplatcn3blob01.blob.core.chinacloudapi.cn",
    "/mnt/bosch-dw-integration-layer",
    {"LinkedService" : "AzureBlobStorage_proddataplatcn3blob01"}
)

StatementMeta(aasparkpool001, 898, 5, Finished, Available)

True

In [5]:
receiver_email = ['external.Zhaoxiang.Wang@cn.bosch.com','fei.zhao2@cn.bosch.com', 'zhaoxiang_wang@leansight.cn'] # set email 
subject = 'Bosch Job Alarm: Travel Expense'
body = 'Travel Expense 报销数据未更新, 请及时确认————This is an alarm email from python. Please do not reply'
body1 = 'Travel Expense 报销数据增加列, 请及时确认————This is an alarm email from python. Please do not reply'

StatementMeta(aasparkpool001, 898, 6, Finished, Available)

In [10]:
print('==================travel expense: Reimbursement=====================')
def get_last_month():
    """ 
     * @ message : 获取上月与本月月份名
     * @ return   [list] 月份
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    previous_month = current_month - 1 if current_month > 1 else 12
    previous_year = current_year if current_month > 1 else current_year -1
    last_month = str(previous_year) + str(previous_month).zfill(2)
    now_month = str(current_year) + str(current_month).zfill(2)
    month_list = [last_month] # remove current month
    return month_list


def data_need(blobs):
    """ 
     * @ message : 获取需要的blob
     * @ param2   [type] blobs: blobs
     * @ return   [type]
    """
    need_blobs = []
    for i in blobs:
        print(i.split('_'))
        split_blob = i.split('_')
        if len(split_blob) > 2:
            blob_name = i.split('_')[-2][:6]
        else:
            blob_name = i
        if blob_name not in get_last_month():
            continue
        need_blobs.append(i)
    return need_blobs

def hand_file():
    path = mssparkutils.fs.getMountPath("/mnt/bosch-dw-integration-layer")
    root = path + '/manual/AA_CTG/Reimbursement'
    files = os.listdir(root)
    files = data_need(files) # enble filter
    try:
        file_path = os.path.join(root,files[0])
        print(file_path)
        return file_path
    except:
        print('=======未识别到上月报销数据=========')
        send_to_email(receiver_email, subject, body)
        mssparkutils.notebook.exit()
file_path = hand_file()

StatementMeta(aasparkpool001, 898, 11, Finished, Available)

==================travel expense: Reimbursement=====================
['Reimbursement', '20230101', '20231130.xlsx']
['Reimbursement', '20231201', '20231231.xlsx']
['WorkON+Platform(16).xlsx']
/synfs/898/mnt/bosch-dw-integration-layer/manual/AA_CTG/Reimbursement/Reimbursement_20231201_20231231.xlsx


In [11]:
def unmerge_and_copy(sheet): # 拆分单元格
    merged_cells = sheet.merged_cells.ranges    # 复制一份合并单元格集合的副本
    for merged_cell in merged_cells.copy():        # 遍历每个合并单元格
        merged_value = sheet.cell(row=merged_cell.min_row, column=merged_cell.min_col).value     # 获取合并单元格的值
        sheet.unmerge_cells(str(merged_cell))    # 拆分合并单元格
        for row in range(merged_cell.min_row, merged_cell.max_row + 1): # 将值分配给每个子单元格
            for column in range(merged_cell.min_col, merged_cell.max_col + 1):
                cell = sheet.cell(row=row, column=column)
                cell.value = merged_value
 
# 共性列名
columns_cols = [
    'Project'
    ,'Key'
    ,'Summary'
    ,'Issue Type'
    ,'Status'
    ,'Priority'
    ,'Resolution'
    ,'Current Assignee'
    ,'Applicant'
    ,'Created'
    ,'Updated'
    ,'Due Date'
    ,'NT-ID Second Level'
    ,'NT-ID First Level'
    ,'Application reason & additional comment'
    ,'Cost Center'
    ,'Company Code'
    ,'Special Approval *'
    ,'Adjustment necessary *'
    ,'Approvers *'
    ,'Advance applied or not *'
    ,'Approved Travel Application'
    ,'Approved Advance Application'
    ,'Charge to other cost center *'
    ,'apcntr.field.trainingProvider'
    ,'apcntr.field.trainingType'
    ,'apcntr.field.trainingNeedsCaused'
    ,'Name (Modify & Post)'
    ,'apcntr.field.thisTrainingCost'
    ,'apcntr.field.tmpTrainingType'
    ,'Open advance'
    ,'Time (Modify & Post)'
    ,'apcntr.field.estimatedTravelExpenses'
    ,'apcntr.field.participantNumber'
    ,'apcntr.field.estimatedOtherCost'
    ,'apcntr.field.estimatedTotalCost'
    ,'common.field.employee.companycode'
    ,'Total - FIN Deduction'
    ,'Total - Kilometers'
    ,'Global CC2'
    ,'Policy Compliance Result'
    ,'Non-compliance Reason'
    ,'Personnel number *'
    ,'Tel.'
    ,'FIN Remark Ref'
    ,'FIN Remark Reason Type'
    ,'FIN Remark Reason Details'
    ,'FIN Remark Reason Code'
    ,'FIN Remark Comment'
    ,'Actual Start Date, Time*:'
    ,'SAP number'
    ,'Charge to Cost Center *'
    ,'Travel or not *'
    ,'Charge to other division (company code)'
    ,'Travel Type'
    ,'Planned End Date:'
    ,'Planned Start Date:'
    ,'Planned Duration:'
    ,'WBS/Internal order Posted'
    ,'Total - Payable'
    ,'Org.Unit'
    ,'Message returned from SAP'
    ,'Actual Duration:'
    ,'Name'
    ,'Actual End Date, Time*:'
    ,'WBS/Internal order'
    ,'Global CC'
    ,'Applicant NT/Display Name'
    ,'Travel to*:'
    ,'travel_allow'
    ,'WBS or Internal order'
    ,'Total - Claim'
    ,'Request need print and delivery?'
    ,'Fapiao & Supporting'
    ,'Total - Actual payment'
    ,'Text sent to SAP *'
]
# 加载Excel文件
workbook = load_workbook(file_path)
sheet = workbook.active
unmerge_and_copy(sheet)

# 处理数据
df_pd = pd.DataFrame(sheet.values,dtype=str)
df_pd.columns = df_pd.iloc[3]
df = df_pd.iloc[4:-1,:]

merge_index = [df.columns.get_loc(col) for col in columns_cols]
for i in range(len(df)):    
    for j in merge_index:  
        if df.iloc[i, j] == '' or df.iloc[i,j] is None: # 如果某个值为空值，则复制上一行此列的数据
            df.iloc[i, j] = df.iloc[i - 1,j]

StatementMeta(aasparkpool001, 898, 12, Finished, Available)

/tmp/ipykernel_8290/1030961550.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i, j] = df.iloc[i - 1,j]


In [12]:
ods_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_reimburse_cn_mf.csv' # path
di_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_reimburse_cn_mi.csv'
formatted_endtime = datetime.now()
formatted_endtime += timedelta(hours=8)
etl_load_time = formatted_endtime.strftime(f"%Y-%m-%d %H:%M:%S")    # etl_load_time

df1 = df
df1['etl_load_time'] = etl_load_time
df1.to_csv(di_path,index=False, header=True)    # 保持数据格式一致性

df1 = pd.read_csv(di_path,dtype=str).drop('etl_load_time',axis=1) 
ods_df = pd.read_csv(ods_path,dtype=str).drop('etl_load_time',axis=1) # read

try:
    result = pd.concat([ods_df, df1]).reset_index(drop=True).drop_duplicates()  # handle
except Exception as e:
    print(f'===========源数据增加列{e}============')
    send_to_email(receiver_email, subject, body1)
    mssparkutils.notebook.exit()

result['etl_load_time'] = etl_load_time

mssparkutils.fs.rm(di_path)
result.to_csv(ods_path,index=False, header=True) # save


StatementMeta(aasparkpool001, 898, 13, Finished, Available)

/tmp/ipykernel_8290/167285046.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['etl_load_time'] = etl_load_time
